In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# Define paths
zip_path = '/content/drive/MyDrive/Colab Notebooks/ASL_Data_Val.zip'
extract_path = '/content/asl'

# Ensure the target directory exists
os.makedirs(extract_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete! Files are stored in:", extract_path)


Extraction complete! Files are stored in: /content/asl


In [ ]:
train_dir = '/content/asl/ASL_Data_Val/train'

labels =['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z','blank','del','space']




In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image
import pathlib
import IPython.display as display

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator




In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2],
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.2  # Splitting dataset
)

# Ensure subset='training' and 'validation' are used correctly
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    subset='training',  # Training subset
    class_mode='categorical'
)

valid_data = datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    subset='validation',  # Validation subset
    class_mode='categorical'
)


Found 15583 images belonging to 29 classes.
Found 3895 images belonging to 29 classes.


In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
from re import I
IMG_SHAPE = IMAGE_SIZE + (3,)
base_model = MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')


In [ ]:
for layer in base_model.layers:
    layer.trainable = False  # Freeze all layers initially
for layer in base_model.layers[-10:]:  # Unfreeze last 10 layers, for example
    layer.trainable = True


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2



# Build model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),  # Reduced dense size
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),  # Reduced dense size
    Dropout(0.3),
    Dense(29, activation='softmax')
])

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
print(model.summary())


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 1280)                │           5,120 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 29)                  │           3,741 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,627,677 (10.02 MB)

 Trainable params: 1,099,613 (4.19 MB)

 Non-trainable params: 1,528,064 (5.83 MB)

None


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

epochs = 15

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    mode='min',
    patience=3,
    factor=0.5,
    verbose=1,
    cooldown=1,
    min_lr=1e-8
)

early_stopper = EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=8,
    verbose=1,
    restore_best_weights=True
)


checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=valid_data,  # Include validation data
    callbacks=[early_stopper, reduce_lr, checkpoint]
)

import matplotlib.pyplot as plt

train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs_range = range(1, len(train_acc) + 1)

for epoch, (train_a, val_a) in enumerate(zip(train_acc, val_acc), 1):
    print(f"Epoch {epoch}: Train Accuracy = {train_a:.4f}, Validation Accuracy = {val_a:.4f}")

plt.figure(figsize=(8, 26))
plt.plot(epochs_range, train_acc, label='Training Accuracy', marker='o')
plt.plot(epochs_range, val_acc, label='Validation Accuracy', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
487/487 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step - accuracy: 0.4591 - loss: 7.7934
Epoch 1: val_loss improved from inf to 5.33344, saving model to best_model.keras
487/487 ━━━━━━━━━━━━━━━━━━━━ 300s 580ms/step - accuracy: 0.4596 - loss: 7.7909 - val_accuracy: 0.8847 - val_loss: 5.3334 - learning_rate: 1.0000e-04
Epoch 2/15
487/487 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step - accuracy: 0.9393 - loss: 4.8692
Epoch 2: val_loss improved from 5.33344 to 3.82180, saving model to best_model.keras
487/487 ━━━━━━━━━━━━━━━━━━━━ 249s 512ms/step - accuracy: 0.9393 - loss: 4.8684 - val_accuracy: 0.9628 - val_loss: 3.8218 - learning_rate: 1.0000e-04
Epoch 3/15
487/487 ━━━━━━━━━━━━━━━━━━━━ 0s 407ms/step - accuracy: 0.9753 - loss: 3.5151
Epoch 3: val_loss improved from 3.82180 to 2.62959, saving model to best_model.keras
487/487 ━━━━━━━━━━━━━━━━━━━━ 247s 506ms/step - accuracy: 0.9753 - loss: 3.5145 - val_accuracy: 0.9815 - val_loss: 2.6296 - learning_rate: 1.0000e-04
Epoch 4/15
487/487 ━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [ ]:
model.evaluate(valid_data)

122/122 ━━━━━━━━━━━━━━━━━━━━ 49s 404ms/step - accuracy: 0.9960 - loss: 0.1699


[0.17144332826137543, 0.995121955871582]

In [ ]:
# prompt: print(model.predict("/content/aslmini/Data/train/N/Image_N_1737740755.1634333.jpg"))

import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = "/content/aslmini/train/E/Image_E_1739374154.2691545.jpg"
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

predictions = model.predict(x)

predicted_class = np.argmax(predictions[0])

print(f"Prediction: {predictions}")

labels =['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
print(f"Predicted Class Label: {labels[predicted_class]}")